In [1]:
from accounts_client import list_accounts_tools, call_accounts_tool, read_accounts_resource

In [ ]:
await list_accounts_tools()

In [ ]:
result = await read_accounts_resource("file://accounts_server/john")
result.contents[0].text

In [ ]:
import requests
from dotenv import load_dotenv
import os
load_dotenv(override=True)
api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=IBM&apikey={api_key}'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
args = {'name': 'john', 'symbol': 'ibm', 'quantity': 1}
await call_accounts_tool('buy_shares', args)

In [ ]:
result = await read_accounts_resource("file://accounts_server/john")
result.contents[0].text

In [ ]:
args = {'name': 'john', 'symbol': 'ibm', 'quantity': 1}
await call_accounts_tool('sell_shares', args)

In [ ]:
result = await read_accounts_resource("file://accounts_server/john")
result.contents[0].text

In [35]:
from agents import FunctionTool

openai_tools = []
import json

tools = await list_accounts_tools()

for tool in tools:
    schema = tool.inputSchema
    schema["additionalProperties"] = False
    openai_tool = FunctionTool(
        name=tool.name,
        description=tool.description,
        params_json_schema=schema,
        on_invoke_tool=lambda ctx, args, toolname=tool.name: call_accounts_tool(toolname, json.loads(args))
            
    )
    openai_tools.append(openai_tool)

openai_tools

[FunctionTool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', params_json_schema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function <lambda> at 0x12a822c00>, strict_json_schema=True),
 FunctionTool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', params_json_schema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function <lambda> at 0x12a820d60>, strict_json_schema=True),
 FunctionTool(name='buy_shares', description='Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The sym

In [41]:
agent_name = "trader"

result = await read_accounts_resource(f"file://accounts_server/{agent_name}")
account_details = result.contents[0].text

instructions = f"""
You are an agent that trades in the stock market. Your name is {agent_name} and your account is under your name, {agent_name}.
Your current account details are:
{account_details}
You have the tools to buy and sell shares. You will be asked to buy or sell particular shares and should use the tools to do so.
If the tools return an error, you should report on the error using natural language.
"""

prompt = "Please execute the following trades to add to the existing portfolio: buy 1 share of IBM"

In [42]:
from agents import Agent, Runner
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=openai_tools,
    model="gpt-4o-mini")

result = await Runner.run(trader, prompt)
print(result.final_output)

The trade to buy 1 share of IBM has been successfully executed. 

### Updated Account Details:
- **New Holdings**: 2 shares of IBM
- **New Balance**: $503.10

If you need further assistance or wish to execute more trades, just let me know!
